In [12]:
import numpy as np
from abc import ABC, abstractmethod

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import simulation as simu
import interference as fere

In [27]:
# get RNG
rng  = np.random.default_rng()

# Time windowns and arrival processes

Define a `TimeWindow` over which to sample, and choose an `ArrivalProcess` process over it.

In [28]:
# make a time window
window = simu.TimeWindow(0, 5, buffer=2)

# specify arrival process
arrivals = simu.PoissonArrivals(window, rate=1.3)

print(arrivals)

HomogeneousPoissonArrivals(TimeWindow(0, 5, buffer=2), rate=1.3)


In [40]:
# sample arrival times
arrs = arrivals.sample(seed=rng)

print(arrs)

[array([ 2.94678576, -1.08423153, -1.78297572,  6.16520756,  3.90314659,
        3.73856414, -1.18942122,  6.03022472,  4.56774225,  0.50053417,
        4.50562599,  6.35223129])]


# Declaring WirelessTraffic

In [33]:
# sample Traffic parameters
nPackets = len(start[0])
airtime = rng.uniform(0.1, 0.2, size=nPackets)
channel = rng.choice(8, size=nPackets)
sf = 7 + rng.choice(4, size=nPackets)
power = rng.normal(loc=-90, scale=10, size=nPackets)

# make traffic object
traffic = fere.Traffic(nPackets, start[0], airtime,
                      channel, sf, power)

# A TrafficGenerator

## Network - A Traffic Generating Process

We keep the `LoRaWAN` class.

Distributionally-equipped traffic-generating networks are children!
- Distribution over channels.
- Distribution over SFs.
- Distribution over payloads(s)

In [18]:
class LoRaParameters():
    """
    An object class for LoRa Network Parameters.

    Attributes
    ---------
    nChannels : int
        Number of channels permitted.

    freq : float
        (Central) frequency used by network.

    bandwidth : float
        Bandwidth of uplink channel.

    spreadingFactors : list of int
        List of spreading factors allowed on network.

    overhead : int
        Number of bytes of overhead in payload.

    dwellTime : float
        Maximum permitted dwell time in ms. Use `None` if not restricted.

    dutyCycle : float
        Maximum duty cycle permitted. Use `None` if not resttrictred.
        

    Notes
    -----
    
    FUTURE - would be nice to read data from e.g.
        https://github.com/TheThingsNetwork/lorawan-frequency-plans

    For now parameters are specified manually.

    Defaults are for North America ISM band.
    """

    def __init__(self, nChannels=1, freq=915, bandwidth=125,
                 spreading=None, overhead=13, maxPow=30,
                 dwellTime=400, dutyCycle=None):
        """
        Instantiate LoRaParemeters instance.

        Parameters
        ----------
        nChannels : int
            Number of channels permitted.

        freq : float
            (Central) frequency used by network.

        bandwidth : float
            Bandwidth of uplink channel.

        spreadingFactors : list of int
            List of spreading factors allowed on network.

        overhead : int
            Number of bytes of overhead in payload.
        
        maxPow : float
            Maximum transmitted power (dBm).

        dwellTime : float
            Maximum permitted dwell time in ms. Use `None` if not restricted.

        dutyCycle : float
            Maximum duty cycle permitted. Use `None` if not resttrictred.
        """
        
        if spreading is None:
            spreading = [7, 8, 9, 10]

        self.nChannels = nChannels
        self.freq = freq
        self.bw = bandwidth
        self.sf = spreading
        self.overhead = overhead
        self.maxPow = maxPow
        self.dwellTime = dwellTime
        self.dutyCycle = dutyCycle

    def __repr__(self):
        return "LoRaParameters(%r)" % self.__dict__

    def __str__(self):
        out = "A LoRa Parameters object with params:"
        for k, v in self.__dict__.items():
            out += f"\n\t {k}: {v}"
        return out

In [19]:
# make LoRaParameters instance
myLoRaWAN = LoRaParameters(nChannels=4)
print(myLoRaWAN)

A LoRa Parameters object with params:
	 nChannels: 4
	 freq: 915
	 bw: 125
	 sf: [7, 8, 9, 10]
	 overhead: 13
	 maxPow: 30
	 dwellTime: 400
	 dutyCycle: None


In [12]:
class LoRaDistribution(LoRaParameters):
    """
    Class of distributions over LoRaWAN parameters.
    May be conditional on arrival times.
    """
    
    


LoRaParameters({'nChannels': 1, 'freq': 915, 'bw': 125, 'sf': [7, 8, 9, 10], 'overhead': 13, 'dwellTime': 400, 'dutyCycle': None})

In [ ]:
class LoRaDistribution()

In [45]:
class LoRaWANProcess(LoRaWAN):
    """
    A LoRaWAN with poisson packet arrivals.
    
    Possibly allow start, stop, buffer, rate to be set separately.
        Then generate an Arrival process internally.
    """
    
    def __init__(self, arrivalProcess, paramDistribution):
        """
        Instantiate a LoRaWAN process.
        
        Parameters
        ----------
        arrivalProcess
             ArrivalProcess-object for sampling packet arrivals.
             This also specifies an observation window.
             
        paramDistribution
             A distribution over LoRaWAN wireless parameters.
             The support of this distribution should be consistent
             with wireless parameters specified by LoRaWAN instance.
             May be conditional on the arrivalProcess.
        
        """
        LoRaWAN.__init__(self)
        
        self.arrivals = arrivalProcess
        self.distribution = paramDistribution
        
    def __repr__(self):
        return "LoRaWANProcess(%r)" % self.__dict__
    
    

In [46]:
wanp = LoRaWANProcess(arrivals, 1)

In [47]:
wanp

LoRaWANProcess({'nChannels': 1, 'freq': 915, 'bw': 125, 'sf': [7, 8, 9, 10], 'overhead': 13, 'dwellTime': 400, 'dutyCycle': None, 'arrivals': HomogeneousPoissonProcess(TimeWindow(0, 5, buffer=2), rate=1.3), 'distribution': 1})

# A Network class

In [ ]:
WAN = mod.LoRaWAN()

print(WAN)

In [ ]:
WAN

## Parameter Distribution

In [ ]:
class distribution(ABC):
    """
    Class for custom made probability distributions.
    
    Notes
    -----
    
    The `__call__` function is used for sampling.
    The parameters and defaults should be explicit.
    Each distribution uses a `numpy.random.generator`,
    and should accept and optional `seed` to which
    a `seed` string or a `numpy.random.generator` can be
    passed.
    
    """
    
    @classmethod
    def __call__(self):
        pass

In [ ]:
class Normal(distribution):
    """A normal distribution.
    """
    
    def __init__(self, loc=0, scale=1):
        """Instantiate normal distribution.
        """
        self.loc = loc
        self.scale = scale
        
    def __repr__(self):
        return "Normal(%s)" % str(self.__dict__) 
        
        
    def __call__(self, loc=None, scale=None, size=None, seed=None):
        """
        Generate a sample of normals.
        """ 
        if loc is None:
            loc = self.loc
        if scale is None:
            scale = self.scale
            
        rng = np.random.default_rng(seed)
        
        return rng.normal(loc=loc, scale=scale, size=size)

In [ ]:
norm = Normal()

In [ ]:
norm(loc=10, scale=5)

In [ ]:
class WirelessDistribution(ABC):
    """
    A distribution over wireless parameters.
    
    Attributes
    ----------
    
    Notes
    -----
    
    This class models any distribution over wireless
    parameters, conditional on arrivals. I.e. the
    number of packets and their arrival times.
    """
    
    @classmethod
    def sample(self):
        pass    

In [ ]:
class Independent(WirelessDistribution):
    """
    A distribution over LoRa Tx parameters in which
    each type of parameter are mutuall independent.
    
    Attributes
    ----------
    payloadDist
        A distribution over payloads.
    channelDist
        A distribution over channels.
    spreadingDidst
        A distribution over spreading factors.
    powerDist
        A distribution over power.
    """
    
    
    

In [ ]:
class LoRaProcess():
    """
    A traffic-generating process.
    
    Attributes
    ----------
    network : LoRaWAN
        A LoRaWAN specifying wireless parameters of traffic.
    arrivals : ArrivalProcess
        Arrival process of wireless traffic.
    paramDist
        A distribution over network parameters.
        
    Notes
    -----
    This is a data-generating process. The main purpose of the class
    is to collect all relevant components in one place.
    
    A general `paramDist` is prefered over a distribution over individual
    wireless parameters, as the more general distribution permits
    arbitrary joint distrtibutions over groups of parameters. E.g. when
    power and SF are correlated.
    """
    
    def __init__(self):
        """
        Instantiate LoRaProcess.
        """
        

In [ ]:
def 